In [1]:
from utils import *
import pickle

In [2]:
CONST_COLUMNS = [
    "Stellensubart_1",
    "Stellensubart_2",
    "Stellensubart_3",
    "Stellensubart_4",
    *[f"T{i}" for i in range(1, 35)],
    *[f"TD{i:02d}" for i in range(1, 35)],
    "Preis",
    "Beleuchtet",
    "Laenge",
    "Breite",
    "Eigenfläche",
    "PPSVACWert",
    "Qid",
    "GJ",
]

In [3]:
data = pd.read_pickle("data/decade_price_data_combined_01_09.pkl")

In [4]:
distance_2018 = pd.read_csv(
    "./data/distances/distances_2018_new_algo_200.csv"
).set_index("Qid_1")
distance_2019 = pd.read_csv(
    "./data/distances/distances_2019_new_algo_200.csv"
).set_index("Qid_1")
distance_2020 = pd.read_csv(
    "./data/distances/distances_2020_new_algo_200.csv"
).set_index("Qid_1")
distance_2021 = pd.read_csv(
    "./data/distances/distances_2021_new_algo_200.csv"
).set_index("Qid_1")
distance_2022 = pd.read_csv(
    "./data/distances/distances_2022_new_algo_200.csv"
).set_index("Qid_1")

In [5]:
# target_year_having_qid = data[data.GJ == 2023].Qid
# cleaned_data = data[data.Qid.isin(target_year_having_qid)]
cleaned_data = data.sort_values(by=["Qid", "GJ"]).set_index("Qid", drop=False)
cleaned_data = cleaned_data.loc[:, CONST_COLUMNS]
# condition = (cleaned_data.reset_index(drop=True).groupby("Qid")["GJ"].apply(lambda x: x.unique().shape[0]) == cleaned_data.reset_index(drop=True).groupby("Qid")["GJ"].count())
# cleaned_data = cleaned_data[cleaned_data.Qid.isin(condition[condition].index)]

# Normalize columns

In [6]:
list_of_t =['TD01', 'TD02', 'TD03', 'TD04',
           'TD05', 'TD06', 'TD07', 'TD08', 'TD09', 'TD10', 'TD11', 'TD12', 'TD13',
           'TD14', 'TD15', 'TD16', 'TD17', 'TD18', 'TD19', 'TD20', 'TD21', 'TD22',
           'TD23', 'TD24', 'TD25', 'TD26', 'TD27', 'TD28', 'TD29', 'TD30', 'TD31',
           'TD32', 'TD33', 'TD34']

for i in list_of_t:
    cleaned_data[i] = (
        cleaned_data[i] - cleaned_data[i].mean()
    ) / cleaned_data[i].std()

cleaned_data.Preis = (
    cleaned_data.Preis - cleaned_data.Preis.mean()
) / cleaned_data.Preis.std()
cleaned_data.Laenge = (
    cleaned_data.Laenge - cleaned_data.Laenge.mean()
) / cleaned_data.Laenge.std()
cleaned_data.Breite = (
    cleaned_data.Breite - cleaned_data.Breite.mean()
) / cleaned_data.Breite.std()
cleaned_data.PPSVACWert = (
    cleaned_data.PPSVACWert - cleaned_data.PPSVACWert.mean()
) / cleaned_data.PPSVACWert.std()

In [7]:
data_2018 = cleaned_data[cleaned_data.GJ == 2018].drop(columns=["Qid", "GJ"]).copy()
data_2019 = cleaned_data[cleaned_data.GJ == 2019].drop(columns=["Qid", "GJ"]).copy()
data_2020 = cleaned_data[cleaned_data.GJ == 2020].drop(columns=["Qid", "GJ"]).copy()
data_2021 = cleaned_data[cleaned_data.GJ == 2021].drop(columns=["Qid", "GJ"]).copy()
data_2022 = cleaned_data[cleaned_data.GJ == 2022].drop(columns=["Qid", "GJ"]).copy()
data_2023 = cleaned_data[cleaned_data.GJ == 2023].drop(columns=["Qid", "GJ"]).copy()

data_2023.loc[:, "T1":"T22"] = data_2023.loc[:, "T1":"T22"].replace(-1, 0)
all_valid_qids = cleaned_data[cleaned_data.Eigenfläche == 1].Qid.unique()
qid_train, qid_val = train_test_split(all_valid_qids, test_size=0.05, random_state=0)

In [8]:
def DataPreprocessor(qids):
    
    MAX_NEIGH = 10
    columns_to_drop = [
        "PPSVACWert",
        *[f"T{i}" for i in range(1, 35)],
        *[f"TD{i:02d}" for i in range(1, 35)],
    ]

    global data_2018, data_2019, data_2020, data_2021, data_2022, data_2023
    global distance_2018, distance_2019, distance_2020, distance_2021, distance_2022
    X, x, y = [], [], []

    all_year_data = [data_2018, data_2019, data_2020, data_2021, data_2022]
    all_year_distances = [
        distance_2018,
        distance_2019,
        distance_2020,
        distance_2021,
        distance_2022,
    ]

    for idx, qid in enumerate(tqdm(qids)):
        
        if (data_2023.index == qid).sum():

            neighbours_features = []

            for year_data, year_distances in zip(all_year_data, all_year_distances):
                current_distances = year_distances[year_distances.index == qid]
                if (current_distances.shape[0] != 0) and ((year_data.index == qid).sum() != 0):
                    current_year_neighbours = current_distances[
                        current_distances.Qid_2 != qid
                    ]
                    
                    current_year_neighbours_data = year_data[year_data.index.isin(current_year_neighbours.Qid_2)].values
#                     torch.from_numpy(
                        
#                     )
                    
#                     current_year_neighbours_data = year_data.loc[current_year_neighbours.Qid2].values
    #                 current_year_neighbours_data_padded = pad(
    #                     current_year_neighbours_data,
    #                     (0, 0, 0, MAX_NEIGH - current_year_neighbours_data.shape[0]),
    #                     "constant",
    #                     0,
    #                 )

                    current_year_self_data = year_data.loc[qid].values

    #                 if (current_year_self_data.ndim == 2) and (
    #                     current_year_self_data.shape[0] > 1
    #                 ):
    #                     current_year_self_data = current_year_self_data[0]

                    current_year_data_point = np.concatenate(
                        [current_year_self_data[None], current_year_neighbours_data],
                        axis=0,
                    )

                else:
                    #TODO: fill dummpy -1 for that Qid, and find that year neighbours.

                    current_year_data_point = np.zeros((1, 78))

                neighbours_features.append(current_year_data_point)


            self_data_2023 = data_2023.loc[qid].drop(labels=columns_to_drop).values

            neighbours_features = np.array(neighbours_features, dtype=object)
            label = torch.tensor(data_2023.loc[qid, "T1":"T22"].mean())

            X.append(neighbours_features)
            x.append(self_data_2023)
            y.append(label)

    return X, x, y

In [9]:
X_train, x_train, y_train = DataPreprocessor(qid_train)

100%|██████████████████████████████████████████████████████████████| 63586/63586 [11:34<00:00, 91.49it/s]


In [10]:
with open("./data/X_train_lstm_new_arch_new_algo_200.pt", "wb") as file_X:
    pickle.dump(X_train, file_X)
with open("./data/x_train_lstm_new_arch_new_algo_200.pt", "wb") as file_x:
    pickle.dump(x_train, file_x)
with open("./data/y_train_lstm_new_arch_new_algo_200.pt", "wb") as file_y:
    pickle.dump(y_train, file_y)

In [11]:
X_val, x_val, y_val = DataPreprocessor(qid_val)

100%|████████████████████████████████████████████████████████████████| 3347/3347 [00:36<00:00, 92.45it/s]


In [12]:
with open("./data/X_val_lstm_new_arch_new_algo_200.pt", "wb") as file_X:
    pickle.dump(X_val, file_X)
    
with open("./data/x_val_lstm_new_arch_new_algo_200.pt", "wb") as file_x:
    pickle.dump(x_val, file_x)
    
with open("./data/y_val_lstm_new_arch_new_algo_200.pt", "wb") as file_y:
    pickle.dump(y_val, file_y)

In [15]:
X_test, x_test, y_test = DataPreprocessor([9860, 9673, 9855, 9333])

100%|██████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 52.42it/s]


In [16]:
with open("./data/X_test_lstm_new_arch_new_algo.pt", "wb") as file_X:
    pickle.dump(X_test, file_X)
    
with open("./data/x_test_lstm_new_arch_new_algo.pt", "wb") as file_x:
    pickle.dump(x_test, file_x)
    
with open("./data/y_test_lstm_new_arch_new_algo.pt", "wb") as file_y:
    pickle.dump(y_test, file_y)

### Combine data with td and old data

In [16]:
old_df = pd.read_pickle("./data/df_gesamt_15_08_prepocessed_einworner_added.pkl")
old_df.drop("Einwohner", inplace=True, axis=1)
new_df = pd.read_pickle("./data/decade_price_data_combined.pkl")
condition = (new_df.reset_index(drop=True).groupby("Qid")["GJ"].apply(lambda x: x.unique().shape[0]) == new_df.reset_index(drop=True).groupby("Qid")["GJ"].count())
new_df = new_df[new_df.Qid.isin(condition[condition].index)]

In [5]:
new_df.columns

Index(['GJ', 'Qid', 'TD01', 'TD02', 'TD03', 'TD04', 'TD05', 'TD06', 'TD07',
       'TD08', 'TD09', 'TD10', 'TD11', 'TD12', 'TD13', 'TD14', 'TD15', 'TD16',
       'TD17', 'TD18', 'TD19', 'TD20', 'TD21', 'TD22', 'TD23', 'TD24', 'TD25',
       'TD26', 'TD27', 'TD28', 'TD29', 'TD30', 'TD31', 'TD32', 'TD33', 'TD34',
       'Ort', 'Preis', 'PLZ', 'Beleuchtet', 'Laenge', 'Breite', 'PPSVACWert',
       'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11',
       'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18', 'T19', 'T20', 'T21',
       'T22', 'T23', 'T24', 'T25', 'T26', 'T27', 'T28', 'T29', 'T30', 'T31',
       'T32', 'T33', 'T34', 'Eigenfläche', 'Stellensubart_1',
       'Stellensubart_2', 'Stellensubart_3', 'Stellensubart_4'],
      dtype='object')

In [6]:
list_of_t = ['TD01', 'TD02', 'TD03', 'TD04', 'TD05', 'TD06', 'TD07',
     'TD08', 'TD09', 'TD10', 'TD11', 'TD12', 'TD13', 'TD14', 'TD15', 'TD16',
     'TD17', 'TD18', 'TD19', 'TD20', 'TD21', 'TD22', 'TD23', 'TD24', 'TD25',
     'TD26', 'TD27', 'TD28', 'TD29', 'TD30', 'TD31', 'TD32', 'TD33', 'TD34', 'Preis']

for i in list_of_t:
    old_df[i] = old_df.Preis

In [7]:
old_df.shape, new_df.shape

((813064, 82), (319594, 82))

In [8]:
combined = pd.concat([new_df, old_df[~(old_df.Qid.isin(new_df.Qid.unique()))]])

In [9]:
combined.shape

(806415, 82)

In [10]:
combined[combined.Qid == 100019.0]

,GJ,Qid,TD01,TD02,TD03,TD04,TD05,TD06,TD07,TD08,...,T30,T31,T32,T33,T34,Eigenfläche,Stellensubart_1,Stellensubart_2,Stellensubart_3,Stellensubart_4
0,2018.0,100019.0,0.000000,0.000,0.000,0.000000,0.000000,0.000,0.000000,6.427,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
52544,2019.0,100019.0,0.650000,0.000,0.000,0.000000,0.000000,0.000,0.000000,0.000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
106327,2020.0,100019.0,0.000000,0.000,0.000,5.967000,0.000000,5.967,0.000000,0.000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
160287,2022.0,100019.0,1.176364,0.000,1.294,6.183636,1.294000,0.000,0.000000,0.000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
213041,2021.0,100019.0,6.427273,0.000,2.156,1.960000,0.715000,0.000,0.000000,0.000,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
268236,2023.0,100019.0,0.000000,1.411,0.000,0.000000,1.282727,0.000,1.282727,0.000,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0,0.0,0.0,0.0


In [14]:
combined.to_pickle("./data/decade_price_data_combined_01_09.pkl")